In [1]:
from __future__ import division

#import torch 
#import torch.nn as nn
#import torch.nn.functional as F 
#from torch.autograd import Variable
import numpy as np

import tensorflow as tf
from tensorflow import Variable
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, BatchNormalization, LeakyReLU, UpSampling2D, Concatenate
from tensorflow.keras.layers import Add, Input
from tensorflow.keras import Model

In [2]:
def parse_cfg(cfgfile):
    """
    Takes a configuration file
    
    Returns a list of blocks. Each blocks describes a block in the neural
    network to be built. Block is represented as a dictionary in the list
    
    """
    
    file = open(cfgfile, 'r')
    lines = file.read().split('\n')                        # store the lines in a list
    lines = [x for x in lines if len(x) > 0]               # get rid of the empty lines 
    lines = [x for x in lines if x[0] != '#']              # get rid of comments
    lines = [x.strip() for x in lines]           # get rid of fringe whitespaces
    
    block = {}
    blocks = []

    for line in lines:
        if line[0] == "[":               # This marks the start of a new block
            if len(block) != 0:          # If block is not empty, implies it is storing values of previous block.
                blocks.append(block)     # add it the blocks list
                block = {}               # re-init the block
            block["type"] = line[1:-1].rstrip()     
        else:
            key,value = line.split("=") 
            block[key.rstrip()] = value.lstrip()
    blocks.append(block)

    return blocks

class DetectionLayer(tf.keras.layers.Layer):
    def __init__(self, anchors, **kwargs):
        super().__init__(**kwargs)
        self.anchors = anchors

def create_modules(input_layer, blocks, num_classes):
    net_info = blocks[0]     #Captures the information about the input and pre-processing    
    module = Sequential()
    layer = input_layer
    prev_filters = 3
    
    outputs = {}
    output_filters = []
    filters = []
    scale = 0
    
    for index, x in enumerate(blocks[1:]):
        
        print(f"index:{index}")

        #check the type of block
        #create a new module for the block
        #append to module_list
        if (x["type"] == "convolutional"):
            
            #Get the info about the layer
            activation = x["activation"]
            try:
                batch_normalize = int(x["batch_normalize"])
                bias = False
            except:
                batch_normalize = 0
                bias = True

            filters= int(x["filters"])
            padding = int(x["pad"])
            kernel_size = int(x["size"])
            stride = int(x["stride"])
            
            print(f"layer{layer.get_shape().as_list()}")

            if padding:
                #model.add(ZeroPadding2D(padding=(kernel_size - 1) // 2))
                layer = ZeroPadding2D(padding=(kernel_size - 1) // 2)(layer)

            #Add the convolutional layer
            
            layer = Conv2D(filters, kernel_size, strides=stride, use_bias = bias, padding="valid",
                           name="conv_{0}".format(index))(layer)
            #model.add(conv, name="conv_{0}".format(index))

            #Add the Batch Norm Layer
            if batch_normalize:
                #model.add(BatchNormalization(), name="batch_norm_{0}".format(index))
                layer = BatchNormalization(name="batch_norm_{0}".format(index))(layer)

            #Check the activation. 
            #It is either Linear or a Leaky ReLU for YOLO
            if activation == "leaky":
                #model.add(LeakyReLU(alpha=0.1), name="leaky_{0}".format(index))
                layer = LeakyReLU(alpha=0.1, name="leaky_{0}".format(index))(layer)

        #If it's an upsampling layer
        #We use Bilinear2dUpsampling
        elif (x["type"] == "upsample"):
            stride = int(x["stride"])
            #model.add(UpSampling2D(size=stride, interpolation="bilinear"), name="upsample_{}".format(index))
            layer = UpSampling2D(size=stride, interpolation="bilinear",
                                 name="upsample_{}".format(index))(layer)
        
        #If it is a route layer
        elif (x["type"] == "route"):
            
            x["layers"] = x["layers"].split(',')
            
            start = int(x["layers"][0])
            if len(x["layers"]) > 1:
                end = int(x["layers"][1]) - index
                filters = output_filters[index + start] + output_filters[end]  # Index negatif :end - index
                #model.add(Concatenate([outputs[index + start], outputs[index + end]], axis=-1), name="route_{}".format(index))
                layer = Concatenate(name="route_{}".format(index))([outputs[index + start], outputs[index + end]])
            else:
                filters = output_filters[index + start]
                #model.add(Concatenate([outputs[index + start]], axis=-1), name="route_{}".format(index))
                #inputs = outputs[index + start]
                #layer = Concatenate([outputs[index + start]], name="route_{}".format(index))(layer)
                layer = outputs[index + start]
        

        #shortcut corresponds to skip connection
        elif x["type"] == "shortcut":
            
            from_ = int(x["from"])
            #inputs = outputs[index - 1] + outputs[index + from_]
            #model.add(Add()([outputs[index - 1], outputs[index + from_]]), name="shortcut_{}".format(index))
            layer = Add(name="shortcut_{}".format(index))([outputs[index - 1], outputs[index + from_]])
            
            #shortcut = EmptyLayer()
            #module.add_module("shortcut_{}".format(index), shortcut)
        
        # Yolo detection layer
        elif x["type"] == "yolo":
            mask = x["mask"].split(",")
            mask = [int(x) for x in mask]

            anchors = x["anchors"].split(",")
            anchors = [int(a) for a in anchors]
            anchors = [(anchors[i], anchors[i+1]) for i in range(0, len(anchors),2)]
            anchors = [anchors[i] for i in mask]

            detection = DetectionLayer(anchors)
            
            #model.add(detection, name="Detection_{}".format(index))
            #layer = Sequential([layer, detection])
            
            n_anchors = len(anchors)
            out_shape = layer.get_shape().as_list()
            layer = tf.reshape(layer, [-1, n_anchors * out_shape[1] * out_shape[2], 5 + num_classes])
            
            box_centers = layer[:, :, 0:2]
            box_shapes = layer[:, :, 2:4]
            confidence = layer[:, :, 4:5]
            classes = layer[:, :, 5:num_classes + 5]
            
            box_centers = tf.sigmoid(box_centers)
            confidence = tf.sigmoid(confidence)
            classes = tf.sigmoid(classes)
            
            anchors = tf.tile(anchors, [out_shape[1] * out_shape[2], 1])
            box_shapes = tf.exp(box_shapes) * tf.cast(anchors, dtype=tf.float32)
            
            x = tf.range(out_shape[1], dtype=tf.float32)
            y = tf.range(out_shape[2], dtype=tf.float32)
            
            cx, cy = tf.meshgrid(x, y)
            cx = tf.reshape(cx, (-1, 1))
            cy = tf.reshape(cy, (-1, 1))
            cxy = tf.concat([cx, cy], axis=-1)
            cxy = tf.tile(cxy, [1, n_anchors])
            cxy = tf.reshape(cxy, [1, -1, 2])
            
            strides = (input_layer.shape[1] // out_shape[1], \
                       input_layer.shape[2] // out_shape[2])
            box_centers = (box_centers + cxy) * strides
            prediction = tf.concat([box_centers, box_shapes, confidence, classes], axis=-1)
            print(f"prediction{prediction.shape}")
            if scale:
                out_pred = tf.concat([out_pred, prediction], axis=1)
            else:
                out_pred = prediction
                scale = 1
                
            print(f"out_pred{out_pred.shape}")
            print(f"current_layer{layer.shape}")
        
        #module_list.add(model)
        #prev_filters = filters
        outputs[index] = layer
        output_filters.append(filters)
        
    #return (net_info, layer)
    model = Model(input_layer, out_pred)
    model.summary()
    return model
    
            
    
    

In [3]:
def load_weights(model, blocks, weightfile):
    # Open the weights file
    fp = open(weightfile, "rb")
    # The first 5 values are header information
    np.fromfile(fp, dtype=np.int32, count=5)
    
    for i, block in enumerate(blocks[1:]):
        if (block["type"] == "convolutional"):
            conv_layer = model.get_layer('conv_' + str(i))
            print("layer: ",i+1,conv_layer)
            filters = conv_layer.filters
            k_size = conv_layer.kernel_size[0]
            in_dim = conv_layer.input_shape[-1]
            if "batch_normalize" in block:
                norm_layer = model.get_layer('batch_norm_' + str(i))
                print("layer: ",i+1,norm_layer)
                size = np.prod(norm_layer.get_weights()[0].shape)
                bn_weights = np.fromfile(fp, dtype=np.float32, count=4 * filters)
                # tf [gamma, beta, mean, variance]
                bn_weights = bn_weights.reshape((4, filters))[[1, 0, 2, 3]]
            else:
                conv_bias = np.fromfile(fp, dtype=np.float32, count=filters)
            # darknet shape (out_dim, in_dim, height, width)
            conv_shape = (filters, in_dim, k_size, k_size)
            conv_weights = np.fromfile(
                fp, dtype=np.float32, count=np.product(conv_shape))
            # tf shape (height, width, in_dim, out_dim)
            conv_weights = conv_weights.reshape(
                conv_shape).transpose([2, 3, 1, 0])
            if "batch_normalize" in block:
                norm_layer.set_weights(bn_weights)
                conv_layer.set_weights([conv_weights])
            else:
                conv_layer.set_weights([conv_weights, conv_bias])
                
    assert len(fp.read()) == 0, 'failed to read all data'
    fp.close()

In [4]:
weightfile = "./weights/yolov3.weights"
cfgfile = "cfg/yolov3.cfg"
input_shape = (416, 416, 3)
num_classes = 80

input_layer=Input(shape=input_shape)

blocks = parse_cfg("./cfg/yolov3.cfg")

model = create_modules(input_layer, blocks, num_classes)

load_weights(model, blocks, weightfile)

try:
    model.save_weights('./weights/yolov3_weights.tf')
    print('\nThe file \'yolov3_weights.tf\' has been saved successfully.')
except IOError:
    print("Couldn't write the file \'yolov3_weights.tf\'.")

index:0
layer[None, 416, 416, 3]
index:1
layer[None, 416, 416, 32]
index:2
layer[None, 208, 208, 64]
index:3
layer[None, 208, 208, 32]
index:4
index:5
layer[None, 208, 208, 64]
index:6
layer[None, 104, 104, 128]
index:7
layer[None, 104, 104, 64]
index:8
index:9
layer[None, 104, 104, 128]
index:10
layer[None, 104, 104, 64]
index:11
index:12
layer[None, 104, 104, 128]
index:13
layer[None, 52, 52, 256]
index:14
layer[None, 52, 52, 128]
index:15
index:16
layer[None, 52, 52, 256]
index:17
layer[None, 52, 52, 128]
index:18
index:19
layer[None, 52, 52, 256]
index:20
layer[None, 52, 52, 128]
index:21
index:22
layer[None, 52, 52, 256]
index:23
layer[None, 52, 52, 128]
index:24
index:25
layer[None, 52, 52, 256]
index:26
layer[None, 52, 52, 128]
index:27
index:28
layer[None, 52, 52, 256]
index:29
layer[None, 52, 52, 128]
index:30
index:31
layer[None, 52, 52, 256]
index:32
layer[None, 52, 52, 128]
index:33
index:34
layer[None, 52, 52, 256]
index:35
layer[None, 52, 52, 128]
index:36
index:37
layer[

layer:  1 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f9788738a30>
layer:  1 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f97885165e0>
layer:  2 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f9788478100>
layer:  2 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f978848ea60>
layer:  3 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f9788498cd0>
layer:  3 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f9788738310>
layer:  4 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f9788516610>
layer:  4 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f9788429d30>
layer:  6 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f9788429310>
layer:  6 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f97884781c0>
layer:  7 <tensorflow.python.keras.

layer:  70 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f9789766a00>
layer:  70 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f97897ca130>
layer:  71 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f9789900ee0>
layer:  71 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f97899008e0>
layer:  73 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f97898e59a0>
layer:  73 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f978bfc9700>
layer:  74 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f9788516730>
layer:  74 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f97882addc0>
layer:  76 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f97882ad460>
layer:  76 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f97897275e0>
layer:  77 <tensorflow.py

In [8]:
blocks = parse_cfg("./cfg/yolov3.cfg")
blocks

[{'type': 'net',
  'batch': '64',
  'subdivisions': '16',
  'width': '608',
  'height': '608',
  'channels': '3',
  'momentum': '0.9',
  'decay': '0.0005',
  'angle': '0',
  'saturation': '1.5',
  'exposure': '1.5',
  'hue': '.1',
  'learning_rate': '0.001',
  'burn_in': '1000',
  'max_batches': '500200',
  'policy': 'steps',
  'steps': '400000,450000',
  'scales': '.1,.1'},
 {'type': 'convolutional',
  'batch_normalize': '1',
  'filters': '32',
  'size': '3',
  'stride': '1',
  'pad': '1',
  'activation': 'leaky'},
 {'type': 'convolutional',
  'batch_normalize': '1',
  'filters': '64',
  'size': '3',
  'stride': '2',
  'pad': '1',
  'activation': 'leaky'},
 {'type': 'convolutional',
  'batch_normalize': '1',
  'filters': '32',
  'size': '1',
  'stride': '1',
  'pad': '1',
  'activation': 'leaky'},
 {'type': 'convolutional',
  'batch_normalize': '1',
  'filters': '64',
  'size': '3',
  'stride': '1',
  'pad': '1',
  'activation': 'leaky'},
 {'type': 'shortcut', 'from': '-3', 'activatio

In [ ]:
https://machinelearningspace.com/yolov3-tensorflow-2-part-2/
    
https://blog.paperspace.com/how-to-implement-a-yolo-v3-object-detector-from-scratch-in-pytorch-part-2/